# Introducción 
La presente investigación tiene como objetivo principal realizar un análisis de datos exhaustivo sobre los delitos ocurridos en la Ciudad de Buenos Aires en el período comprendido entre los años 2016 y 2021. Este proyecto busca explorar y comprender en profundidad el panorama delictivo en una de las ciudades más grandes y emblemáticas de América Latina.

La importancia de esta investigación radica en la vasta cantidad de información recopilada y puesta a disposición por parte de la Ciudad de Buenos Aires. La disponibilidad de datos confiables y detallados sobre los delitos cometidos en la ciudad proporciona una oportunidad invaluable para comprender las tendencias delictivas a lo largo del tiempo, identificar patrones, y formular estrategias de prevención y seguridad pública basadas en evidencia.

#  Contexto - Preguntas de Negocio
La recopilación y gestión de datos sobre delitos es fundamental para la toma de decisiones informadas por parte de las autoridades y la comunidad en general. La Ciudad de Buenos Aires ha demostrado su compromiso con la transparencia y la rendición de cuentas al proporcionar acceso a esta información, lo que permite a investigadores, analistas y ciudadanos interesados profundizar en el estudio de la seguridad urbana y colaborar en la construcción de soluciones efectivas.

A lo largo de este documento, se realizará un análisis detallado de los datos disponibles, se explorarán las variaciones en la incidencia delictiva a lo largo de los años y se identificarán los delitos más comunes en diferentes áreas de la ciudad. Además, se pretende evaluar cómo ciertos eventos o circunstancias pueden haber influido en los patrones de delincuencia, por ejemplo el efecto Covid-19.

En resumen, este proyecto de análisis de datos sobre delitos en la Ciudad de Buenos Aires no solo busca informar sobre la situación delictiva, sino también destacar el esfuerzo y la importancia de la Ciudad en la recopilación y accesibilidad de estos datos, contribuyendo así a la toma de decisiones más informadas y a la mejora de la calidad de vida de sus habitantes.

## Preguntas a responder.
1. Cual es el Barrio con mayor incidencia de delitos. Indicador promedio de delitos mensual. 
2. Durante la pandemia el numero de delitos semanales disminuyo? En que magnitud? Analisis en el Top 5 de barrios inseguros de la pregunta 1.
3. Todos los dias de la semana tienen el mayor riesgo en cuando a la incidencia de delitos?
4. Cuales son las horas del dia con mayor numero de delitos?

# Propuesta de Solución - Desarrollo

A continuación se presenta el desarrollo de la resolución explicando el código desarrollado.

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.graph_objects as go
import os
import sqlite3

### Set de Datos Disponibles

In [4]:
link_2016 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2016.csv"
link_2017 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2017.csv"
link_2018 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2018.csv"
link_2019 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-justicia-y-seguridad/delitos/delitos_2019.csv"
link_2020 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/mapa-del-delito/delitos_2020.csv"
link_2021 = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-justicia-y-seguridad/delitos/delitos_2021.csv"

In [21]:
# pd.read_csv(link_2019, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2019.csv')
# pd.read_csv(link_2018, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2018.csv')
# pd.read_csv(link_2017, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2017.csv')
# pd.read_csv(link_2016, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']].to_csv('./data/delitos_2016.csv')
# pd.read_csv(link_2021, sep=';').loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']].to_csv('/data/delitos_2021.csv')
#pd.read_csv(link_2020, sep=',', low_memory=False).loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']]#.to_csv('./data/delitos_2020.csv')

In [5]:
links =[link_2016, link_2017, link_2018,link_2019, link_2020, link_2021]


### Extracción, Carga y Transformación

Se define una función de extracción, transformación y carga para respaldar los datos de manera física.

1. Creación de Carpeta de Almacenamiento:

+ La función `os.mkdir()` se utiliza para crear una carpeta (directorio) llamada "datalake" en el directorio actual (donde se encuentra el script de Python) para almacenar los archivos resultantes. Si la carpeta ya existe, se manejará la excepción `FileExistsError`.

2. Bucle para Procesar Archivos:

+ Se inicia un bucle for para recorrer una lista de enlaces (links) que se supone que contienen los datos de los delitos correspondientes a diferentes años.

+ Se extrae el año del enlace analizando el nombre del archivo. Esto se hace dividiendo el nombre del archivo por "_", tomando los primeros 4 caracteres después del primer guion bajo (por ejemplo, "delitos_2016.csv" daría como resultado "2016").

+ Luego, se intenta leer y procesar cada archivo CSV de acuerdo al año extraído.

3. Procesamiento de Datos por Año:

* Para los años 2016 a 2019:

    + Los archivos se leen utilizando pd.read_csv() con un separador de coma (",") y se seleccionan las columnas específicas ('fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long').
    + La columna 'fecha' se convierte en objetos de fecha y hora utilizando pd.to_datetime().
    + Los datos se guardan en un archivo CSV en la carpeta "datalake" con un nombre que incluye el año correspondiente, por ejemplo, "delitos_2016.csv".
    + Los datos se agregan a una lista de DataFrames (dfs) para su posterior concatenación.

* Para el año 2020:

    + Los archivos se leen con un separador de coma (",") y se seleccionan las mismas columnas que en los años anteriores.
    + Se renombran las columnas para que coincidan con las de años anteriores ('fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long').
    + La columna 'fecha' se convierte en objetos de fecha y hora utilizando pd.to_datetime(), especificando el formato correcto ("%d/%m/%Y").
    + Los datos se guardan en un archivo CSV en la carpeta "datalake" con un nombre que incluye el año correspondiente, por ejemplo, "delitos_2020.csv".
    + Los datos se agregan a la lista dfs.

* Para el año 2021:

    + Los archivos se leen con un separador de punto y coma (";") y se seleccionan las mismas columnas que en los años anteriores.
    + Se renombran las columnas para que coincidan con las de años anteriores ('fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long').
    + La columna 'fecha' se convierte en objetos de fecha y hora utilizando pd.to_datetime().
    + Los datos se guardan en un archivo CSV en la carpeta "datalake" con un nombre que incluye el año correspondiente, por ejemplo, "delitos_2021.csv".
    + Los datos se agregan a la lista dfs.

4. Consolidación de Datos:

+ Finalmente, se utiliza pd.concat() para combinar todos los DataFrames almacenados en dfs en un solo DataFrame llamado "consolidado". Esto une todos los datos de los diferentes años en uno solo.

+ El DataFrame consolidado se guarda en un archivo CSV llamado "delitos_consolidado.csv" en la carpeta "datalake".

5. Resultado:

+ La función devuelve el DataFrame consolidado que contiene todos los datos de delitos de los años procesados.

En resumen, este código se encarga de leer archivos CSV de diferentes años, procesar los datos y consolidarlos en un solo archivo CSV. También crea una carpeta llamada "datalake" para almacenar los archivos resultantes. Además, maneja posibles errores durante el proceso de lectura y procesamiento de datos.

In [19]:
def extract_load(links):
    ruta_carpeta = './datalake'
    # Utiliza la función mkdir() para crear la carpeta
    try:
        os.mkdir(ruta_carpeta)
        print(f'Se ha creado la carpeta en {ruta_carpeta}')
    except FileExistsError:
        print(f'La carpeta {ruta_carpeta} ya existe.')
    except Exception as e:
        print(f'Error al crear la carpeta: {str(e)}')
        
    dfs = []
    for link in links:
        anio = int(link.split("_")[1][0:4])
        try:
            if anio in [2016, 2017, 2018, 2019] :
                #print(f"procesando datos de {anio}")
                data = pd.read_csv(link, sep=',', low_memory=False).loc[:,['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']]
                data["fecha"] = pd.to_datetime(data["fecha"])
                data.to_csv(f'./datalake/delitos_{anio}.csv')
                dfs.append(data)
            elif anio == 2020:
                data = pd.read_csv(link_2020, sep=',', low_memory=False).loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']]
                data.columns = ['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']
                data["fecha"] = pd.to_datetime(data["fecha"], format="%d/%m/%Y")
                data.to_csv(f'./datalake/delitos_{anio}.csv')
                dfs.append(data)
            elif anio == 2021:
                data = pd.read_csv(link_2021, sep=';',low_memory=False).loc[:,['fecha', 'franja', 'tipo', 'barrio', 'comuna', 'latitud', 'longitud']]
                data.columns = ['fecha', 'franja_horaria', 'tipo_delito', 'barrio', 'comuna', 'lat', 'long']
                data["fecha"] = pd.to_datetime(data["fecha"])
                data.to_csv(f'./datalake/delitos_{anio}.csv')
                dfs.append(data)
            else:
                print("El dataset no esta en los años con disponibilidad de delitos")
        except Exception as e:
            # Manejar la excepción, por ejemplo, imprimir un mensaje de error
            print(f"No se pudo leer el enlace {link}: {str(e)}")
    consolided = pd.concat(dfs, ignore_index=True)
    consolided.to_csv(f'./datalake/delitos_consolidado.csv')
    return consolided

In [20]:
extract_load(links)

La carpeta ./datalake ya existe.


,fecha,franja_horaria,tipo_delito,barrio,comuna,lat,long
0,2016-01-01,13,Robo (con violencia),Parque Avellaneda,9.0,-34.6487,-58.4685
1,2016-01-01,22,Robo (con violencia),Palermo,14.0,-34.5769,-58.4074
2,2016-01-01,1,Lesiones,Parque Patricios,4.0,-34.6452,-58.4011
3,2016-01-01,19,Hurto (sin violencia),Parque Avellaneda,9.0,-34.6558,-58.4787
4,2016-01-01,4,Robo (con violencia),Caballito,6.0,-34.6241,-58.4368
...,...,...,...,...,...,...,...
648640,2021-08-05,10,Hurto (sin violencia),Balvanera,3.0,"-34,605751","-58,395375"
648641,2021-06-29,21,Robo (con violencia),Palermo,14.0,"-34,59266","-58,422338"
648642,2021-06-30,13,Robo (con violencia),Balvanera,3.0,"-34,608791","-58,404617"
648643,2021-07-31,1,Hurto (sin violencia),Balvanera,3.0,"-34,603284","-58,394526"


### Exploración y Limpieza de Datos

Una vez respaldado el dataset compilado se lo carga como variables para comenzar el trabajo de exploración y análisis.

In [26]:
delitos = pd.read_csv("./datalake/delitos_consolidado.csv",  low_memory=False).drop("Unnamed: 0", axis=1)
delitos["fecha"] = pd.to_datetime(delitos["fecha"])

In [27]:
def safe_float_conversion(value):
    try:
        return float(value.replace(',', '.'))
    except (ValueError, AttributeError):
        return np.nan

In [28]:
delitos["anio"] = delitos['fecha'].dt.strftime('%Y')
delitos["mes"] = delitos['fecha'].dt.strftime('%m')
delitos["semana"] = delitos['fecha'].dt.strftime('%W')
delitos["dia"] = delitos['fecha'].dt.strftime('%d')
delitos["dia_semana"] = delitos['fecha'].dt.day_name()

delitos["lat"] = delitos["lat"].apply(safe_float_conversion)
delitos["long"] = delitos["long"].apply(safe_float_conversion)
delitos["franja_horaria"] = delitos["franja_horaria"].apply(safe_float_conversion)
delitos['periodo'] = 'PrePandemia'
delitos.loc[delitos['fecha'] >= '2020-03-01', 'periodo'] = 'PostPandemia'


In [29]:
delitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648645 entries, 0 to 648644
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           648645 non-null  datetime64[ns]
 1   franja_horaria  646551 non-null  float64       
 2   tipo_delito     648645 non-null  object        
 3   barrio          638598 non-null  object        
 4   comuna          639115 non-null  float64       
 5   lat             638598 non-null  float64       
 6   long            638598 non-null  float64       
 7   anio            648645 non-null  object        
 8   mes             648645 non-null  object        
 9   semana          648645 non-null  object        
 10  dia             648645 non-null  object        
 11  dia_semana      648645 non-null  object        
 12  periodo         648645 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(8)
memory usage: 64.3+ MB


El dataset tiene un total de 648.645 filas y 7 columnas.

In [31]:
delitos.shape

(648645, 13)

La cantidad de datos nulos es despresiable respecto del total de filas. Por lo cual se decide eliminar los registros nulos.

In [32]:
delitos.isna().sum()

fecha                 0
franja_horaria     2094
tipo_delito           0
barrio            10047
comuna             9530
lat               10047
long              10047
anio                  0
mes                   0
semana                0
dia                   0
dia_semana            0
periodo               0
dtype: int64

In [33]:
delitos = delitos.dropna()

In [35]:
#vamos a discretizar los horarios
# Armamos los (rangos)
bins = [-1, 6, 12, 20, 23]#ponemos -1 para que tome el valor 0
# nombre de cada rango
labels = ['Madrugada', 'Mañana', 'Tarde', 'Noche']
delitos["horario"] = pd.cut(delitos.franja_horaria, bins = bins, labels = labels)

In [36]:
delitos.sample(10)

,fecha,franja_horaria,tipo_delito,barrio,comuna,lat,long,anio,mes,semana,dia,dia_semana,periodo,horario
4811,2016-01-16,22.0,Robo (con violencia),Paternal,15.0,-34.601084,-58.459606,2016,01,02,16,Saturday,PrePandemia,Noche
567655,2021-12-30,23.0,Robo (con violencia),Caballito,6.0,-34.609689,-58.450640,2021,12,52,30,Thursday,PostPandemia,Noche
141974,2017-02-17,15.0,Hurto (sin violencia),Palermo,14.0,-34.578214,-58.402608,2017,02,07,17,Friday,PrePandemia,Tarde
544450,2020-12-09,15.0,Hurto (sin violencia),Balvanera,3.0,-34.610041,-58.404580,2020,12,49,09,Wednesday,PostPandemia,Tarde
214014,2017-09-27,17.0,Robo (con violencia),Floresta,10.0,-34.626764,-58.491125,2017,09,39,27,Wednesday,PrePandemia,Tarde
302549,2018-06-14,21.0,Robo (con violencia),Flores,7.0,-34.632521,-58.462699,2018,06,24,14,Thursday,PrePandemia,Noche
336501,2018-09-24,12.0,Robo (con violencia),San Nicolás,1.0,-34.603060,-58.383676,2018,09,39,24,Monday,PrePandemia,Mañana
401448,2019-05-10,22.0,Hurto (sin violencia),Saavedra,12.0,-34.552263,-58.488528,2019,05,18,10,Friday,PrePandemia,Noche
45171,2016-05-12,10.0,Lesiones,Villa Gral. Mitre,11.0,-34.616751,-58.466763,2016,05,19,12,Thursday,PrePandemia,Mañana
439539,2019-01-01,12.0,Lesiones,Flores,7.0,-34.649498,-58.442125,2019,01,00,01,Tuesday,PrePandemia,Mañana


In [37]:
ts = delitos.groupby(["fecha"], as_index=False)["tipo_delito"].count().set_index('fecha').dropna()

### Exploración de Datos

La pandemia de COVID-19 ha tenido un impacto significativo en la comisión y la tipología de delitos en todo el mundo. A continuación, se presentan algunas el comportamiento de los delitos a lo largo del tiempo para la Ciudad de Buenos Aires.

Se puede observar una caida en la ocurrencia de delitos durante la pandemia.

In [38]:
fig = px.line()

# Calcular la media móvil de 10 periodos
ts['media_movil'] = ts['tipo_delito'].rolling(window=50).mean()

# Agregar la serie de tiempo con transparencia
fig.add_trace(go.Scatter(x=ts.index, y=ts['tipo_delito'], mode='lines', name='Delitos Seria Temporal Diaria',
                        line=dict(color='blue'), opacity=0.4))  # Color azul con transparencia

# Agregar la media móvil con un color distinguible
fig.add_trace(go.Scatter(x=ts.index, y=ts['media_movil'], mode='lines', name='Media Móvil (50 períodos)',
                         line=dict(color='red')))  # Color rojo


# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Fecha', yaxis_title='Cantidad de Delitos',legend=dict(x=0, y=1))
fig.show()

El código y gráfico anterios utiliza la biblioteca Plotly Express para crear un gráfico de serie de tiempo con una media móvil de 50 períodos y personaliza su apariencia. A continuación, se explican los pasos:

+ fig = px.line(): Esto crea una figura vacía para el gráfico de línea utilizando Plotly Express. La figura se almacena en la variable fig y se usará para construir el gráfico.

+ ts['media_movil'] = ts['tipo_delito'].rolling(window=50).mean(): Calcula la media móvil de 50 períodos de la columna 'tipo_delito' y la almacena en una nueva columna llamada 'media_movil' en el DataFrame ts. Esto suaviza la serie de tiempo original.

+ fig.add_trace(go.Scatter(...)): Agrega trazos al gráfico. En este caso, se agregan dos trazos (líneas).

+ El primer trazo representa la serie de tiempo original de delitos diarios. Utiliza el modo 'lines' para trazar líneas entre los puntos de datos. El color de las líneas se establece en azul y se agrega cierta transparencia (0.4) para que el gráfico sea más legible.

+ El segundo trazo representa la media móvil calculada. Utiliza el modo 'lines' para trazar líneas entre los puntos de datos. El color de las líneas se establece en rojo.

+ fig.update_layout(...): Personaliza el diseño del gráfico. En este caso:

+ xaxis_title y yaxis_title establecen los títulos de los ejes X e Y, respectivamente.
+ legend controla la posición de la leyenda en el gráfico, con coordenadas (x, y). En este caso, se coloca en la esquina superior derecha.
+ fig.show(): Muestra el gráfico resultante en la salida.

En resumen, este código utiliza Plotly Express para crear un gráfico de serie de tiempo que muestra los datos de delitos diarios junto con una media móvil de 50 períodos. El gráfico resultante tiene dos líneas: una en azul (para los datos originales) y otra en rojo (para la media móvil), con una leyenda en la esquina superior derecha y títulos en los ejes X e Y.

### Respuestas de Negocio.
1. Cual es el Barrio con mayor incidencia de delitos. Indicador promedio de delitos mensual. 
2. Durante la pandemia el numero de delitos semanales disminuyo? En que magnitud? Analisis en el Top 5 de barrios inseguros de la pregunta 1.
3. Todos los dias de la semana tienen el mayor riesgo en cuando a la incidencia de delitos?
4. Cuales son las horas del dia con mayor numero de delitos?

In [39]:
ruta_carpeta = './sql'
# Utiliza la función mkdir() para crear la carpeta
try:
    os.mkdir(ruta_carpeta)
    print(f'Se ha creado la carpeta en {ruta_carpeta}')
except FileExistsError:
    print(f'La carpeta {ruta_carpeta} ya existe.')
except Exception as e:
    print(f'Error al crear la carpeta: {str(e)}')

La carpeta ./sql ya existe.


In [40]:
conn = sqlite3.connect('./sql/delitos.db')

In [41]:
delitos.to_sql("delitos", conn, if_exists= "replace", index= False)

636587

#### 1. Promedios de Delitos Mensuales - Barrio

In [24]:
consulta1 = 'select barrio, round(avg(numeroDelitosMes),0 )as promedioMensual, min(numeroDelitosMes), max(numeroDelitosMes) from(select barrio, anio, mes, count(*) as numeroDelitosMes from delitos d group by barrio, mes, anio order by barrio, anio,mes)as resumen group by barrio order by promedioMensual desc;'

In [43]:
con1 = pd.read_sql_query(consulta1, conn)
con1.head()

,barrio,promedioMensual,min(numeroDelitosMes),max(numeroDelitosMes)
0,Palermo,718.0,113,1010
1,Balvanera,615.0,120,981
2,Flores,463.0,115,648
3,Recoleta,418.0,95,570
4,Caballito,412.0,82,591


In [45]:
fig = go.Figure()

fig.add_trace(go.Bar(x=con1["barrio"], y=con1["min(numeroDelitosMes)"], name="Mínimo",
                     marker_color="blue"))
fig.add_trace(go.Bar(x=con1["barrio"], y=con1["promedioMensual"], name="Promedio",
                     marker_color="green"))
fig.add_trace(go.Bar(x=con1["barrio"], y=con1["max(numeroDelitosMes)"], name="Máximo",
                     marker_color="red"))

fig.update_layout(title="Mínimo, Promedio y Máximo de Delitos por Barrio",
                  xaxis_title="Barrio", yaxis_title="Cantidad de Delitos",
                  barmode="relative", height=500)

fig.show()

#### 2. Analisis en el Top 5 de barrios inseguros de la pregunta 1.

In [49]:
#2. Durante la pandemia el numero de delitos semanales disminuyo? En que magnitud? Analisis en el Top 5 de barrios inseguros de la pregunta 1.
consulta2 =  "select barrio, anio, round(avg(numeroDelitoSemana), 0) as promedioSemanal from(select barrio, anio,  semana, count(*) as numeroDelitoSemana from delitos d where barrio in ('Palermo', 'Balvanera', 'Flores', 'Recoleta', 'Caballito') group by barrio, semana, anio) as resumensemana group by barrio, anio order by barrio, anio;"


In [51]:
con2 = pd.read_sql_query(consulta2, conn)
con2

,barrio,anio,promedioSemanal
0,Balvanera,2016,173.0
1,Balvanera,2017,134.0
2,Balvanera,2018,155.0
3,Balvanera,2019,179.0
4,Balvanera,2020,84.0
5,Balvanera,2021,110.0
6,Caballito,2016,119.0
7,Caballito,2017,108.0
8,Caballito,2018,104.0
9,Caballito,2019,97.0


In [68]:
fig = px.bar(con2, x='anio', y='promedioSemanal', text_auto='.2s', facet_col= "barrio",
            title="Controlled text sizes, positions and angles")
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title="Promedio Semanal de Delitos por Barrio",
                  xaxis_title="Año", yaxis_title="Promedio Semanal de Delitos",
                  barmode="relative", height=500)
fig.show()

#### 3. Todos los dias de la semana tienen el mayor riesgo en cuando a la incidencia de delitos?

In [58]:
consulta3 = "select dia_semana, round(avg(numeroDelitosDiaSemana),1) as promedioDiario, min(numeroDelitosDiaSemana), max(numeroDelitosDiaSemana)from(select  anio, semana as semana, dia_semana, count(*) as numeroDelitosDiaSemana from delitos d group by  semana, dia_semana, anio order by  anio, semana )as resumen group by dia_semana order by 2 desc;"


In [59]:
con3 = pd.read_sql_query(consulta3, conn)
con3

,dia_semana,promedioDiario,min(numeroDelitosDiaSemana),max(numeroDelitosDiaSemana)
0,Friday,309.9,41,453
1,Wednesday,307.6,47,486
2,Thursday,305.2,36,448
3,Tuesday,302.7,25,465
4,Monday,302.3,36,482
5,Saturday,273.5,40,441
6,Sunday,231.6,28,467


In [66]:
fig = px.bar(con3, x="dia_semana", y="promedioDiario", title="Wide-Form Input")
fig.update_xaxes(categoryorder='array', categoryarray=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
fig.show()

#### 4. Cuales son las horas del dia con mayor numero de delitos?

In [71]:
consulta4= "select horario, round(avg(numerodelitosdia),1) as promediodiario from(select  anio, mes, semana as dia,horario, count(*) as numeroDelitosDia from delitos d group by mes, dia, anio, horario order by  anio, mes, dia)as resumen group by horario order by 2 desc;"


In [75]:
con4 = pd.read_sql_query(consulta4, conn)
con4

,horario,promediodiario
0,Tarde,130.3
1,Mañana,72.4
2,Madrugada,45.0
3,Noche,42.8


In [77]:
fig = px.bar(con4, x="horario", y="promediodiario", title="Delitos y Horarios")
fig.update_xaxes(categoryorder='array', categoryarray=["Madrugada","Mañana", "Tarde", "Noche"])
fig.show()

# Concluisiones

La pandemia de COVID-19 ha tenido un impacto significativo en la comisión y la tipología de delitos en todo el mundo. A continuación, se presentan algunas de las consecuencias más comunes de la pandemia en la comisión de delitos:

* Cambio en los Patrones de Delincuencia:

Durante las fases más críticas de la pandemia, se observó una disminución en los delitos tradicionales como robos y hurtos, ya que las medidas de confinamiento y el cierre de negocios limitaron las oportunidades para la delincuencia callejera.

* Aumento en Delitos en Línea:

Con más personas trabajando desde casa y realizando compras en línea, hubo un aumento en los delitos cibernéticos, incluyendo estafas en línea, fraudes y ataques informáticos.
Violencia Doméstica:

Se informó un aumento en los casos de violencia doméstica durante la pandemia, ya que las personas pasaron más tiempo en sus hogares y algunas quedaron atrapadas en situaciones de abuso.
Delitos Económicos:

Se produjo un aumento en los delitos económicos relacionados con la crisis económica causada por la pandemia, como el fraude financiero y el robo de identidad.

* Cierre de Bares y Restaurantes:

El cierre de bares y restaurantes en muchas áreas dio lugar a una disminución en la delincuencia relacionada con la vida nocturna, como la violencia en bares y los accidentes de tráfico relacionados con el alcohol.
Presión sobre el Sistema de Justicia Penal:

La pandemia ejerció presión sobre el sistema de justicia penal, con retrasos en los juicios y la liberación anticipada de algunos reclusos para reducir la densidad en las cárceles.


* Cambios en la Política de Aplicación de la Ley:

En algunos lugares, las autoridades ajustaron las políticas de aplicación de la ley para enfocarse en hacer cumplir las restricciones de salud pública en lugar de delitos menores.

* Desafíos en la Recopilación de Datos:

La pandemia planteó desafíos en la recopilación de datos sobre delitos debido a interrupciones en la presentación de denuncias y la operación de las fuerzas del orden.
Es importante destacar que el impacto de la pandemia en la comisión de delitos puede variar según la ubicación geográfica y la fase de la pandemia. Además, las políticas y las respuestas gubernamentales desempeñaron un papel crucial en la forma en que se manifestaron estas tendencias delictivas.